In [6]:
study = 'Sleep Lab'

### DB and S3 ###

In [21]:
from aws_manager import S3Manager, DBManagerSystem
import yaml
import pandas as pd

def fetch_motion_results(db_manager, session_id):
    query = f"SELECT * FROM MotionResult WHERE SessionID = {session_id}"
    return pd.read_sql(query, db_manager.engine)

def fetch_study_sessions(db_manager, study):
    query = f"""
    SELECT Patient.PatientStudyName, Patient.EnrollmentTime,
    Session.* FROM Session 
    JOIN Patient ON Session.PatientID = Patient.ID 
    JOIN Study ON Patient.StudyID = Study.ID 
    WHERE Study.Name = '{study}'
    """
    return pd.read_sql(query, db_manager.engine)


def load_config():
    with open('parameters.yaml', 'r') as file:
        config = yaml.safe_load(file)
    return config

S3_CONFIG = load_config()['Cloud']['s3']
DB_CONFIG = load_config()['Cloud']['DB_system']

s3_manager = S3Manager(
    access_key_id=S3_CONFIG['access_key_id'],
    secret_access_key=S3_CONFIG['secret_access_key'],
    region=S3_CONFIG['region'],
    bucket_name=S3_CONFIG['bucket_name']
)

db_manager = DBManagerSystem(
    host=DB_CONFIG['host'],
    user=DB_CONFIG['user'],
    password=DB_CONFIG['password'],
    database=DB_CONFIG['database']
)

In [15]:
import numpy as np
session_df = fetch_study_sessions(db_manager, study)
session_df.head()

session_ids = session_df['ID'].tolist()
print(session_ids)

[299, 375, 379, 380, 381, 382, 384, 385, 386]


In [36]:
import os
import os
import pandas as pd
import numpy as np
from mne.io import read_raw_edf

class SleepLabProcessor:
    def __init__(self, db_manager, s3_manager):
        self.db_manager = db_manager
        self.s3_manager = s3_manager
        self.local_path = 'data'
        if not os.path.exists(self.local_path):
            os.makedirs(self.local_path)

    def process(self, session_id, num_samples):
        self.session_id = session_id
        self.fetch_patient_info()
        edf_file_path = self.download_edf_file()
        self.print_channels_and_save_chest(edf_file_path)
        motion_results = self.fetch_motion_results()
        self.choose_samples_and_download_signals(motion_results, num_samples)

    def fetch_patient_info(self):
        query = f"""
        SELECT Patient.ID, Patient.PatientStudyName FROM Session 
        JOIN Patient ON Session.PatientID = Patient.ID
        WHERE Session.ID = {self.session_id}
        """
        result = pd.read_sql(query, self.db_manager.engine)
        if not result.empty:
            self.patient_id = result.iloc[0]['ID']
            self.patient_name = result.iloc[0]['PatientStudyName']
        else:
            raise ValueError("Session ID not found")

    def download_edf_file(self):
        self.patient_number = self.patient_name.replace('SL', '')
        edf_local = f"{self.local_path}/{self.patient_number}.edf"
        edf_s3 = f"{self.s3_manager.bucket}/{self.patient_number}/{edf_local}"
        print(f"edf s3 path: {edf_s3}")
        print(f"edf local path: {edf_local}")
        local_path = os.path.join(self.local_path, edf_local)
        self.s3_manager.download_from_s3(edf_s3, local_path)
        return local_path

    def print_channels_and_save_chest(self, file_path):
        raw = read_raw_edf(file_path, preload=True)
        print("Channels:", raw.ch_names)
        
        # Assuming 'Chest' is the label for the chest channel
        if 'Chest' in raw.ch_names:
            chest_data = raw.get_data(picks='Chest')
            timestamps = raw.times
            
            # Save chest signal with timestamps
            chest_df = pd.DataFrame({'Timestamp': timestamps, 'Chest': chest_data[0]})
            chest_df.to_csv('/local/path/to/save/chest_data.csv', index=False)
        else:
            print("Chest channel not found.")

    def fetch_motion_results(self):
        query = f"SELECT * FROM MotionResult WHERE SessionID = {self.session_id}"
        return pd.read_sql(query, self.db_manager.engine)

    def choose_samples_and_download_signals(self, motion_results, num_samples):
        session_start = motion_results['StartTime'].min()
        session_end = motion_results['EndTime'].max()
        session_duration = (session_end - session_start).total_seconds()
        
        sample_times = np.linspace(session_start, session_end, num_samples)
        
        for time in sample_times:
            closest_signal = motion_results.iloc[(motion_results['RadarSignalID'] - time).abs().argsort()[:1]]
            if closest_signal['MotionType'].iloc[0] == 'Mild':
                radar_signal_id = closest_signal['RadarSignalID'].iloc[0]
                self.download_signal(radar_signal_id)

    def download_signal(self, radar_signal_id):
        signal_filename = f"signal_{radar_signal_id}.dat"
        local_path = os.path.join("/local/path/to/save", signal_filename)
        self.s3_manager.download_file(signal_filename, local_path)
        print(f"Downloaded signal: {signal_filename}")

In [37]:
session_id = session_ids[0]
processor = SleepLabProcessor(db_manager=db_manager, s3_manager=s3_manager)
processor.process(session_id=session_id, num_samples=10)

edf s3 path: sleep-lab/0001/0001.edf
edf local path: 0001.edf


ClientError: An error occurred (404) when calling the HeadObject operation: Not Found

In [ ]:
sig_pp = sig - np.mean(sig, axis=0)
max_ind = np.argmax(np.std(sig_pp, axis=0))
resp_signal = sig_pp[:, max_ind]